In [ ]:
import glob
from datetime import datetime
import os
from multiprocessing import Pool

import numpy as np
from scipy.ndimage import gaussian_filter
from tqdm.notebook import tqdm

from uncertainty import ComputeSidSicUncertainty
%matplotlib inline

In [ ]:
n_steps = 2210 # should be 2210 to cover 6 years

sia_dir = 'data2/Anton/sia/cdr_1991_2023'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/unc'
sid_dir = 'data2/Anton/sia_sid_cdr_postproc'
ifiles = sorted(glob.glob(f'{sid_dir}/*/ice_drift_nh_ease2*.nc.npz'))
idates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M%S')
          for ifile in ifiles]

mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'
xc = np.load(mesh_init_file)['xc']
yc = np.load(mesh_init_file)['yc']
mask = np.load(mesh_init_file)['mask']
mask_f = gaussian_filter(mask.astype(float), 1, truncate=1)

In [ ]:
start_indices = [i for i,j in enumerate(idates) if j.year > 2011 and j.month == 9 and j.day == 5]

In [ ]:
for start_idx in start_indices:
    stop_idx = min(start_idx + n_steps, len(idates))
    print(f'Processing {start_idx} to {stop_idx} for {idates[start_idx]} to {idates[stop_idx-1]}')
    comp = ComputeSidSicUncertainty(ifiles, idates, idates[start_idx], sia_dir, mesh_dir, mesh_init_file, xc, yc)
    with Pool(4) as p:
        results = list(tqdm(
            p.imap(comp.compute_sid_sic_uncertainty, range(start_idx, stop_idx)),
            total=stop_idx - start_idx
            ))
